In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pretty_midi

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio

In [ ]:
import io
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import sys
from decimal import Decimal, ROUND_HALF_UP
sys.path.append('/content/drive/MyDrive/s_melody_generation')
from utils.transformer import TransformerClassification
from utils.torch import DataLoader

In [ ]:
poslist1 = dict()
barlist1=["Position_0","Position_1","Position_2","Position_3","Position_4","Position_5","Position_6","Position_7","Position_8","Position_9","Position_10","Position_11","Position_12","Position_13","Position_14","Position_15","Position_16","Position_17","Position_18","Position_19","Position_20","Position_21","Position_22","Position_23","Position_24","Position_25","Position_26","Position_27","Position_28","Position_29","Position_30","Position_31","Bar_None"]
for i in range(31):
  poslist1[i]=[]
  for j in range(31-i):
    poslist1[i].append("Position_"+str(i+j+1))
  poslist1[i].append("Bar_None")
poslist2 = dict()
barlist2=[0.8966118813699382, 0.00036727573225599127, 0.010100082637039761, 0.0006427325314479846, 0.05660637223395466, 0.0013313745294279684, 0.0008263703975759807, 0.00013772839959599677, 0.013543292626939683, 4.590946653199891e-05, 0.00018363786612799564, 4.590946653199891e-05, 0.0019281975943439547, 0.0, 0.0, 4.590946653199891e-05, 0.005095950785051879, 0.0, 0.00022954733265999458, 0.0, 0.001469102929023965, 4.590946653199891e-05, 4.590946653199891e-05, 0.0, 0.0018822881278119554, 0.0, 0.00022954733265999458, 0.0, 0.0010100082637039764, 4.590946653199891e-05, 0.00027545679919199355, 4.590946653199891e-05, 0.00720778624552383]
poslist2[0]=[0.020279289454769485, 0.09456823082451997, 0.012270253619468117, 0.5377862203511654, 0.012064893726255263, 0.03886435979053291, 0.000872779546154636, 0.22723072184002463, 0.00015401991990964166, 0.0007187596262449944, 0.0005647397063353527, 0.018071670602731287, 0.0005647397063353527, 0.0003080398398192833, 0.00020535989321285552, 0.023565047746175172, 5.133997330321388e-05, 0.0, 0.0, 0.0011294794126707054, 0.0005647397063353527, 0.00020535989321285552, 0.0, 0.002721018585070336, 0.0, 0.00010267994660642776, 0.0, 0.0006160796796385666, 0.0, 5.133997330321388e-05, 5.133997330321388e-05, 0.006417496662901735]
poslist2[1]=[0.5796019900497512, 0.1517412935323383, 0.14676616915422885, 0.009950248756218905, 0.004975124378109453, 0.0, 0.06467661691542288, 0.0, 0.0, 0.0, 0.0024875621890547263, 0.0, 0.004975124378109453, 0.0, 0.017412935323383085, 0.0, 0.0, 0.0, 0.0024875621890547263, 0.0, 0.0, 0.0, 0.004975124378109453, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009950248756218905]
poslist2[2]=[0.08467917939764295, 0.6979484941073767, 0.006547359231776516, 0.11086861632474902, 0.0008729812309035355, 0.07682234831951112, 0.0008729812309035355, 0.0026189436927106066, 0.0, 0.0030554343081623746, 0.0, 0.00043649061545176777, 0.0, 0.006110868616324749, 0.0, 0.0, 0.0, 0.00043649061545176777, 0.0, 0.00043649061545176777, 0.0, 0.00043649061545176777, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00785683107813182]
poslist2[3]=[0.4671968190854871, 0.34592445328031807, 0.09343936381709742, 0.0019880715705765406, 0.06759443339960239, 0.0019880715705765406, 0.0019880715705765406, 0.005964214711729622, 0.009940357852882704, 0.0, 0.0, 0.0, 0.003976143141153081, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[4]=[0.014862776837613126, 0.22853358840409094, 0.004046795673607534, 0.712236038554926, 0.00022073430946950188, 0.002943124126260025, 0.00022073430946950188, 0.030019866087852256, 7.357810315650063e-05, 0.00014715620631300125, 0.0, 0.005150467220955044, 0.0, 0.0, 0.0, 0.0011036715473475094, 0.0, 0.0, 0.0, 7.357810315650063e-05, 0.0, 0.0, 0.0, 0.00014715620631300125, 7.357810315650063e-05, 0.0, 0.0, 0.00014715620631300125]
poslist2[5]=[0.21580547112462004, 0.06534954407294834, 0.6063829787234044, 0.007598784194528875, 0.006079027355623099, 0.04103343465045593, 0.01519756838905775, 0.027355623100303948, 0.0, 0.0, 0.007598784194528875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0015197568389057749, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006079027355623099]
poslist2[6]=[0.02983348751156337, 0.7821461609620722, 0.0009250693802035154, 0.07631822386679002, 0.0025439407955596674, 0.10198889916743757, 0.00023126734505087885, 0.00208140610545791, 0.0, 0.0025439407955596674, 0.0, 0.00023126734505087885, 0.00023126734505087885, 0.0006938020351526365, 0.0, 0.00023126734505087885, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[7]=[0.916, 0.028, 0.004, 0.008, 0.028, 0.0, 0.0, 0.004, 0.008, 0.0, 0.0, 0.0, 0.004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[8]=[0.011058027806108754, 0.13892318428257017, 0.016372322722636747, 0.6424392076869396, 0.017714316388426642, 0.01266842020505663, 0.00091255569273713, 0.15003489183531052, 5.367974663159588e-05, 0.00026839873315797946, 0.0, 0.002791346824842986, 0.0003220784797895753, 0.00016103923989478766, 0.0, 0.0035428632776853286, 5.367974663159588e-05, 0.00021471898652638353, 0.0, 0.00016103923989478766, 0.0, 0.00010735949326319177, 0.0, 0.0022008696118954313]
poslist2[9]=[0.44396551724137934, 0.1810344827586207, 0.24568965517241378, 0.02586206896551724, 0.01293103448275862, 0.004310344827586207, 0.08189655172413793, 0.0, 0.0, 0.0, 0.004310344827586207, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[10]=[0.058557101261727584, 0.7405370430281462, 0.006793917825946294, 0.15108379165318664, 0.0006470397929472662, 0.038498867680362335, 0.0, 0.0012940795858945324, 0.0, 0.0009705596894208991, 0.0, 0.0009705596894208991, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0006470397929472662]
poslist2[11]=[0.4709897610921502, 0.30887372013651876, 0.07337883959044368, 0.005119453924914676, 0.12457337883959044, 0.0017064846416382253, 0.0017064846416382253, 0.0, 0.006825938566552901, 0.0034129692832764505, 0.0, 0.0, 0.0017064846416382253, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0017064846416382253]
poslist2[12]=[0.018797703904623733, 0.20147606131331613, 0.0025231817321642596, 0.6590550684413046, 0.00037847725982463897, 0.005992556613890117, 0.00031539771652053244, 0.09316848546016528, 0.000126159086608213, 0.000504636346432852, 0.0, 0.01148047688134738, 6.30795433041065e-05, 6.30795433041065e-05, 0.0, 0.0017662272125149817, 0.0, 0.00018923862991231948, 6.30795433041065e-05, 0.004037090771462816]
poslist2[13]=[0.23529411764705876, 0.04498269896193771, 0.6747404844290656, 0.004613610149942329, 0.004613610149942329, 0.0011534025374855823, 0.012687427912341403, 0.0023068050749711646, 0.005767012687427912, 0.0011534025374855823, 0.012687427912341403, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[14]=[0.03338938265673793, 0.7948594763391785, 0.0012010569300984868, 0.11097766034110017, 0.000720634158059092, 0.04083593562334855, 0.0, 0.00360317079029546, 0.0, 0.007926975738650012, 0.0, 0.0, 0.0, 0.002161902474177276, 0.0, 0.00024021138601969732, 0.0, 0.004083593562334855]
poslist2[15]=[0.8780487804878049, 0.0040650406504065045, 0.08130081300813008, 0.008130081300813009, 0.012195121951219513, 0.0, 0.0040650406504065045, 0.0040650406504065045, 0.008130081300813009, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
poslist2[16]=[0.011969472354911328, 0.13149947839455334, 0.015099105034865206, 0.5778839290616593, 0.01910723109866579, 0.022456487124581342, 0.0011530225662987976, 0.18613078570252017, 0.00016471750947125678, 0.00032943501894251357, 0.0007686817108658651, 0.008784933838467028, 0.0004941525284137704, 0.0004392466919233515, 0.00010981167298083787, 0.023609509690880142]
poslist2[17]=[0.5635593220338984, 0.1271186440677966, 0.1652542372881356, 0.029661016949152543, 0.0211864406779661, 0.0, 0.07627118644067797, 0.0, 0.0, 0.0, 0.00423728813559322, 0.0, 0.0, 0.0, 0.012711864406779662]
poslist2[18]=[0.05318808074335149, 0.7526433835309195, 0.0067286126241589235, 0.11887215636014098, 0.0003204101249599487, 0.05767382249279077, 0.0003204101249599487, 0.003524511374559436, 0.0009612303748798462, 0.003524511374559436, 0.0, 0.0, 0.0, 0.002242870874719641]
poslist2[19]=[0.4585062240663901, 0.4232365145228216, 0.06431535269709544, 0.006224066390041494, 0.03526970954356847, 0.0, 0.0020746887966804984, 0.004149377593360997, 0.0020746887966804984, 0.0, 0.0, 0.0, 0.004149377593360997]
poslist2[20]=[0.019494875058618612, 0.19253701346553226, 0.0030816640986132513, 0.7392644201782006, 0.0003349634889797012, 0.0029476787030213707, 6.699269779594024e-05, 0.03523815904066457, 6.699269779594024e-05, 0.00013398539559188047, 0.0, 0.006833255175185905]
poslist2[21]=[0.22793296089385476, 0.049162011173184354, 0.5988826815642458, 0.0033519553072625698, 0.0067039106145251395, 0.049162011173184354, 0.013407821229050279, 0.004469273743016759, 0.0033519553072625698, 0.0, 0.04357541899441341]
poslist2[22]=[0.039165818921668365, 0.8044252288911495, 0.000508646998982706, 0.08545269582909461, 0.005849440488301119, 0.05035605289928789, 0.000762970498474059, 0.004577822990844354, 0.0, 0.008901322482197355]
poslist2[23]=[0.8708487084870848, 0.02214022140221402, 0.01107011070110701, 0.033210332103321034, 0.02214022140221402, 0.0, 0.01107011070110701, 0.007380073800738007, 0.02214022140221402]
poslist2[24]=[0.013353037420172058, 0.14081384915817807, 0.020794848788726446, 0.5998838866311289, 0.01752256293872381, 0.009658521137911016, 0.000422230432258405, 0.19755106349290125]
poslist2[25]=[0.5127272727272727, 0.17454545454545456, 0.17454545454545456, 0.05090909090909091, 0.01090909090909091, 0.007272727272727273, 0.06909090909090909]
poslist2[26]=[0.07437248218159281, 0.7964053300278897, 0.007437248218159281, 0.07096374341493647, 0.0006197706848466068, 0.05020142547257515]
poslist2[27]=[0.44587628865979384, 0.3234536082474227, 0.029639175257731958, 0.002577319587628866, 0.19845360824742267]
poslist2[28]=[0.022581489723785837, 0.16815028145045163, 0.0023563293624820005, 0.8069118994632806]
poslist2[29]=[0.20710659898477157, 0.047715736040609136, 0.7451776649746192]
poslist2[30]=[0.041268863566368955, 0.9587311364336311]
ma1=["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50"]
ma2=[0.0, 0.0, 0.3861397190578863, 0.0, 0.25609677286693733, 0.0, 0.16498152315040568, 0.0, 0.10271248377033482, 0.0, 0.05775704558406232, 0.0, 0.025350590142938553, 0.0, 0.004200619224148567, 0.0, 0.001744872600800174, 0.0, 0.0004112494345320275, 0.0, 0.00021149970918789987, 0.0, 8.22498869064055e-05, 0.0, 0.00012924982228149437, 0.0, 0.00011749983843772214, 0.0, 2.349996768754443e-05, 0.0, 1.1749983843772215e-05, 0.0, 1.1749983843772215e-05, 0.0, 1.1749983843772215e-05, 0.0, 0.0, 0.0, 5.874991921886108e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
pitchlist1=["21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74","75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90","91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106","107","108"]
durationlist1=["Duration_0.1.8","Duration_0.2.8","Duration_0.3.8","Duration_0.4.8","Duration_0.5.8","Duration_0.6.8","Duration_0.7.8","Duration_1.0.8","Duration_1.1.8","Duration_1.2.8","Duration_1.3.8","Duration_1.4.8","Duration_1.5.8","Duration_1.6.8","Duration_1.7.8","Duration_2.0.8","Duration_2.1.8","Duration_2.2.8","Duration_2.3.8","Duration_2.4.8","Duration_2.5.8","Duration_2.6.8","Duration_2.7.8","Duration_3.0.8","Duration_3.1.8","Duration_3.2.8","Duration_3.3.8","Duration_3.4.8","Duration_3.5.8","Duration_3.6.8","Duration_3.7.8","Duration_4.0.4","Duration_4.1.4","Duration_4.2.4","Duration_4.3.4","Duration_5.0.4","Duration_5.1.4","Duration_5.2.4","Duration_5.3.4","Duration_6.0.4","Duration_6.1.4","Duration_6.2.4","Duration_6.3.4","Duration_7.0.4","Duration_7.1.4","Duration_7.2.4","Duration_7.3.4","Duration_8.0.4","Duration_8.1.4","Duration_8.2.4","Duration_8.3.4","Duration_9.0.4","Duration_9.1.4","Duration_9.2.4","Duration_9.3.4","Duration_10.0.4","Duration_10.1.4","Duration_10.2.4","Duration_10.3.4","Duration_11.0.4","Duration_11.1.4","Duration_11.2.4","Duration_11.3.4","Duration_12.0.4"]
durationlist2=[0.033193, 0.219399, 0.0177748, 0.4132287, 0.0047593, 0.0355191, 0.0015354, 0.1700712, 0.0034433, 0.0020787, 0.0243629, 0.0054352, 0.0008264, 0.000579, 0.0320299, 0.0058867, 0.0012447, 0.0007983, 0.0030607, 0.0004208, 0.0002141, 0.0009386, 0.0068814, 0.0005815, 0.0002831, 0.0002244, 0.0008085, 0.0004055, 0.0001071, 0.0075726, 1.28e-05, 0.0005484, 0.0008876, 5.87e-05, 0.0012982, 4.08e-05, 0.0001454, 7.14e-05, 0.000732, 7.91e-05, 2.55e-05, 0.0005637, 4.08e-05, 7.4e-05, 2.6e-06, 0.0008672, 7.7e-06, 0.0002168, 5.1e-06, 0.0001454, 0.0, 1.79e-05, 0.0, 7.91e-05, 0.0, 7.7e-06, 0.0, 8.67e-05, 2.6e-06, 0.0, 9.95e-05, 2.6e-06, 2.3e-05, 0.0001938]
token_idx = {'Duration_10.3.4': 185, 'Duration_10.0.4': 184, 'Duration_9.0.4': 182, 'Duration_11.3.4': 180, 'Duration_3.7.8': 179, 'Duration_4.2.4': 173, 'Duration_5.0.4': 170, 'Duration_11.1.4': 167, 'Pitch_105': 165, 'Duration_9.2.4': 164, 'Pitch_102': 163, 'Pitch_104': 172, 'Duration_10.2.4': 162, 'Duration_3.5.8': 159, 'Duration_12.0.4': 158, 'Duration_5.1.4': 157, 'Duration_8.0.4': 156, 'Pitch_108': 178, 'Duration_8.2.4': 155, 'Pitch_101': 154, 'Pitch_107': 177, 'Duration_2.5.8': 153, 'Pitch_23': 152, 'Duration_3.2.8': 150, 'Duration_3.1.8': 149, 'Duration_3.4.8': 148, 'Pitch_100': 147, 'Position_31': 140, 'Duration_3.0.8': 139, 'Pitch_22': 166, 'Position_9': 138, 'Position_15': 135, 'Duration_6.1.4': 176,
 'Position_7': 131, 'Duration_3.3.8': 130, 'Duration_1.6.8': 129, 'Duration_7.1.4': 186, 'Duration_1.5.8': 127, 'Pitch_49': 57, 'Pitch_61': 48, 'Pitch_58': 55, 'Pitch_63': 50, 'Pitch_43': 47, 'Pitch_79': 40, 'Duration_6.3.4': 168, 'Pitch_51': 53, 'Position_0': 8, 'Pitch_77': 46, 'Pitch_85': 84, 'Position_4': 17, 'Duration_0.2.8': 6, 'Pitch_34': 76, 'Pitch_78': 39, 'Duration_0.3.8': 56, 'Pitch_68': 42, 'Pitch_47': 44, 'Pitch_70': 34, 'Pitch_73': 33, 'Duration_7.0.4': 171, 'Pitch_57': 31, 'Duration_8.1.4': 183, 'Position_29': 104, 'Pitch_52': 30, 'Duration_4.0.4': 146, 'Pitch_62': 29, 'Duration_1.0.8': 7, 'Pitch_48': 37, 'bansou': 99, 'Duration_5.2.4': 174, 'Position_24': 11, 'Position_8': 10, 'Bar_None': 9,
 'Duration_2.4.8': 141, 'Position_20': 14, 'Pitch_76': 19, 'Pitch_45': 35, 'Pitch_53': 51, 'Position_10': 77, 'Position_18': 79, 'Duration_1.7.8': 18, 'Pitch_75': 54, 'Position_16': 12, 'Duration_0.6.8': 15, 'Duration_6.0.4': 161, '100': 4, 'Pitch_59': 38, 'Duration_7.3.4': 181, 'Position_23': 137, '<pad>': 1, 'Pitch_99': 151, 'Position_22': 70, 'Pitch_94': 128, 'Pitch_88': 89, 'Pitch_72': 25, 'Duration_0.4.8': 5, 'Pitch_103': 160, 'Pitch_60': 41, 'Position_30': 72, 'Pitch_46': 43, 'Position_12': 13, 'Duration_6.2.4': 142, 'Pitch_55': 32, 'Pitch_24': 145, 'Pitch_69': 20, 'Duration_4.3.4': 114, 'Pitch_74': 21, 'Position_3': 116, 'Duration_1.3.8': 26, 'Pitch_67': 23, 'Position_26': 82, 'Pitch_82': 71, '<unk>': 0, 'Pitch_56': 58, 'Pitch_54': 52, 'Pitch_65': 49,
 'Pitch_89': 101, 'Position_1': 115, 'Pitch_66': 36, 'Pitch_71': 24, 'Pitch_50': 28, 'Pitch_83': 60, 'Pitch_41': 61, 'Duration_5.3.4': 134, 'Pitch_33': 81, 'Position_11': 113, 'Pitch_106': 169, 'Pitch_44': 63, 'Pitch_40': 64, 'Position_6': 65, 'sabi': 92, 'Pitch_64': 27, 'Position_27': 110, 'Position_28': 16, 'Pitch_39': 69, 'Pitch_28': 107, 'Pitch_25': 143, 'Duration_0.1.8': 22, 'Pitch_36': 68, 'Position_14': 67, 'Pitch_84': 73, 'Pitch_35': 74, 'Duration_2.1.8': 117, 'Position_13': 105, 'Pitch_80': 59, 'Pitch_37': 75, 'Duration_3.6.8': 78, 'Duration_2.7.8': 83, 'Position_17': 136, 'Pitch_98': 132, 'Pitch_86': 80, 'Duration_2.0.8': 86, 'Pitch_81': 45, '<cls>': 2, 'Pitch_31': 88, 'abcmero': 90, 'Duration_1.2.8': 102, 'Position_2': 87, 'Pitch_32': 93, 'Pitch_87': 94,
 'Pitch_21': 175, 'Pitch_30': 95, 'Position_5': 108, 'Duration_2.3.8': 96, 'Pitch_96': 125, 'Duration_1.1.8': 97, 'Pitch_29': 98, 'Pitch_97': 144, 'Duration_1.4.8': 85, 'Pitch_91': 100, 'Position_21': 106, 'Position_25': 133, 'Duration_0.7.8': 109, 'Pitch_93': 111, 'Duration_0.5.8': 91, 'Pitch_27': 121, 'Pitch_38': 62, 'Pitch_92': 112, 'Pitch_26': 118, 'Pitch_95': 119, 'Duration_2.6.8': 120, 'Position_19': 122, 'Duration_2.2.8': 123, 'Pitch_90': 103, '<eos>': 3, 'Pitch_42': 66, 'Duration_7.2.4': 124, 'Duration_4.1.4': 126}


In [ ]:
input_dim=len(token_idx)
# モデル構築
net = TransformerClassification(
    input_dim=input_dim, d_model=512, max_seq_len=256, output_dim=2)

In [ ]:
#学習済みモデルの読み込み
#CPU
#net.load_state_dict(torch.load('/content/drive/MyDrive/s_melody_generation/model_cpu.pth'))

In [ ]:
#学習済みモデルの読み込み
#GPU
net.load_state_dict(torch.load('/content/drive/MyDrive/s_melody_generation/model_gpu.pth'))

In [ ]:
#トークン列をID列に変換
def vocab(sentence):
  global token_idx
  output=[[]]
  output[0].append(token_idx['<cls>'])
  for w in sentence:
    if w not in token_idx:
      idx=token_idx['<unk>']
    else:
      idx=token_idx[w]
    output[0].append(idx)
  output[0].append(token_idx['<eos>'])
  return output

In [ ]:
def evaluate(s):
  global tempmax
  global txt
  if len(s)>254:
    for i in range(len(s)-254):
      s.pop(-1)
  elif len(s)<254:
    for i in range(254-len(s)):
      s.append("100")
  output=vocab(s)
  label=[1]
  data_loader = DataLoader((output, label))
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  net.eval()
  net.to(device)
  for batch in (data_loader):
    inputs = batch[0].to(device)
    labels = batch[1].to(device)
    with torch.set_grad_enabled(False):
      input_pad = 4
      input_mask = (inputs != input_pad)
      outputs, _, _ = net(inputs, input_mask)
  if tempmax>(outputs[0][0]-outputs[0][1]):
    tempmax=outputs[0][0]-outputs[0][1]
    txt=s
    print(outputs)

In [ ]:
#初期メロディの作成
with io.open('/content/drive/MyDrive/s_melody_generation/data/pitch_data.txt','r',encoding="utf-8")as f:
  texttemp=f.readlines()
textselection1 = random.randint(0,len(texttemp)-1)
s1=texttemp[textselection1].replace('[',"").replace(']',"").replace(',',"").replace('\'',"").replace('\n',"").split(' ')
pitchlist2 = [0 for i in range(88)]
for i in range(len(s1)):
  for j in range(88):
    if "Pitch_"+str(21+j) == s1[i]:
      pitchlist2[j]+=1
      break
total1=sum(pitchlist2)
for i in range(88):
  pitchlist2[i]=(pitchlist2[i]/total1)
  pitchlist2[i]=Decimal(str(pitchlist2[i])).quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP)
total2=sum(pitchlist2)
for i in range(88):
  if pitchlist2[i]!=0:
    a=i
    break
pitchlist2[a]=pitchlist2[a]+(1-total2)

textselection2 = random.randint(0,len(texttemp)-1)
s2=texttemp[textselection2].replace('[',"").replace(']',"").replace(',',"").replace('\'',"").replace('\n',"").split(' ')
temp=1
patern = []
patern.append(s2[0])
patern.append(s2[1])
while True:
  if temp>255:
    break
  elif patern[temp]=="Bar_None":
    temp+=1
    patern.append(np.random.choice(barlist1, p=barlist2))
  else:
    for i in range(32):
      if patern[temp]=="Position_"+str(i):
        ranma=int(np.random.choice(ma1, p=ma2))
        for j in range(int((ranma-1)/2)):
          patern.append("Pitch_"+np.random.choice(pitchlist1, p=pitchlist2))
          patern.append(np.random.choice(durationlist1,p=durationlist2))
        temp=temp+ranma
        if patern[temp-ranma]=="Position_31":
          patern.append("Bar_None")
        else:
          patern.append(np.random.choice(poslist1[i], p=poslist2[i]))
        break
if len(patern)>255:
  for i in range(len(patern)-255):
    patern.pop(-1)
#patern:初期メロディ

#初期メロディを良いメロディに変換
tempmax=100
txt=patern
for i in range(500):
  s=[target for target in txt if target!="100" ]
  length=len(s)
  choice=np.random.choice(["置換","追加","削除","pos追加"],p=[0.25,0.25,0.25,0.25])
  countbar=0
  barichi, sentaku, sentakus=[], [], []
  if choice=="pos追加":
    ran="Pitch_"+np.random.choice(pitchlist1, p=pitchlist2)
    ran2=np.random.choice(durationlist1,p=durationlist2)
    for j in range(length):
      if "Position_" in s[j] or "Bar_" in s[j]:
        sentaku.append(j)
        sentakus.append(s[j])
        if "Bar_" in s[j]:
          countbar+=1
          barichi.append(sentaku.index(j))
    while True:
      tempcountbar=random.randint(0,countbar-1)
      tempran=np.random.choice(["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"], p=[0.11468478027987564, 0.0023599016519297447, 0.013459609421718545, 0.0029799020859296774, 0.07979765585831139, 0.003869902708929582, 0.025379217765437258, 0.001470001028999841, 0.10938697657081817, 0.0013600009519998532, 0.018129512690648043, 0.003439902407929628, 0.09305736514009993, 0.0050898035628594495, 0.024449317114507357, 0.0014500010149998432, 0.10702707491888842, 0.00139000097299985, 0.018329512830648015, 0.002829901980929695, 0.08757756130424053, 0.005249803674859432, 0.02308931616250751, 0.0015900011129998282, 0.11127697789381798, 0.001610001126999826, 0.01893941325757795, 0.004559903191929507, 0.08987746291417029, 0.005779804045859376, 0.01910941337657793, 0.0014000009799998486])
      if tempcountbar==countbar-1:
        if "Position_"+str(tempran) in sentakus[barichi[tempcountbar]:]:
          continue
        else:
          for j in range(int(tempran),32):
            for k in range(barichi[tempcountbar],len(sentakus)):
              if "Position_"+str(j) == sentakus[k]:
                s[sentaku[k]:sentaku[k]] = ["Position_"+str(tempran), ran, ran2]
                break
            else:
              continue
            break
          break
      else:
        if "Position_"+str(tempran) in sentakus[barichi[tempcountbar]:barichi[tempcountbar+1]]:
          continue
        else:
          for j in range(int(tempran),32):
            for k in range(barichi[tempcountbar],barichi[tempcountbar+1]):
              if "Position_"+str(j) == sentakus[k]:
                s[sentaku[k]:sentaku[k]] = ["Position_"+str(tempran), ran, ran2]
                break
              if j==31:
                s[sentaku[barichi[tempcountbar+1]]:sentaku[barichi[tempcountbar+1]]] = ["Position_"+str(tempran), ran, ran2]
                break
            else:
              continue
            break
          break
    evaluate(s)
  if choice=="削除":
    if length<120:
      break
    for j in range(length):
      if "Duration_" in s[j]:
        sentaku.append(j)
    temp=random.choice(sentaku)
    s.pop(temp-1)
    s.pop(temp-1)
    if len(s)!=temp-1:
      if ("Position_" in s[temp-2] and "Position_" in s[temp-1]) or ("Position_" in s[temp-2] and "Bar_" in s[temp-1]):
        s.pop(temp-2)
        length-=1
    evaluate(s)
  elif choice=="追加":
    for j in range(length):
      if "Pitch_" in s[j]:
        if "Duration_" in s[j-1]:
          continue
        else:
          sentaku.append(j)
      elif "Duration_" in s[j]:
        sentaku.append(j+1)
    temp=random.choice(sentaku)
    ran="Pitch_"+np.random.choice(pitchlist1, p=pitchlist2)
    ran2=np.random.choice(durationlist1,p=durationlist2)
    s[temp:temp] = [ran, ran2]
    evaluate(s)
  elif choice=="置換":
    for j in range(length):
      if "Pitch_" in s[j] or "Duration_" in s[j]:
        sentaku.append(j)
    temp=random.choice(sentaku)
    if "Pitch_" in s[temp]:
      s[temp]="Pitch_"+np.random.choice(pitchlist1, p=pitchlist2)
    elif "Duration" in s[temp]:
      s[temp]=np.random.choice(durationlist1,p=durationlist2)
    evaluate(s)

In [ ]:
import pretty_midi
previous_dura=["0.09","0.15","0.2","0.25","0.3","0.35","0.4","0.5","0.55","0.6","0.65","0.75","0.8","0.85","0.9","1.0","1.05","1.1","1.15","1.2","1.3","1.35","1.4","1.45","1.55","1.6","1.65","1.75","1.85","1.9","1.95","2.0","2.1","2.2","2.35","2.45","2.55","2.7","2.8","2.95","3.05","3.2","3.3","3.5","3.6","3.8","3.9","4.0","4.1","4.3","4.4","4.5","4.6","4.7","4.9","5.0","5.1","5.2","5.4","5.5","5.6","5.7","5.8","6.0"]
back_dura=["0.1.8","0.2.8","0.3.8","0.4.8","0.5.8","0.6.8","0.7.8","1.0.8","1.1.8","1.2.8","1.3.8","1.4.8","1.5.8","1.6.8","1.7.8","2.0.8","2.1.8","2.2.8","2.3.8","2.4.8","2.5.8","2.6.8","2.7.8","3.0.8","3.1.8","3.2.8","3.3.8","3.4.8","3.5.8","3.6.8","3.7.8","4.0.4","4.1.4","4.2.4","4.3.4","5.0.4","5.1.4","5.2.4","5.3.4","6.0.4","6.1.4","6.2.4","6.3.4","7.0.4","7.1.4","7.2.4","7.3.4","8.0.4","8.1.4","8.2.4","8.3.4","9.0.4","9.1.4","9.2.4","9.3.4","10.0.4","10.1.4","10.2.4","10.3.4","11.0.4","11.1.4","11.2.4","11.3.4","12.0.4"]
cello_c_chord = pretty_midi.PrettyMIDI(initial_tempo=120)
cello = pretty_midi.Instrument(0)
temppos1, temppos2 = 0, 0
s=[target for target in txt if target!="100" ]
s[0]="Bar_None"
for i in range(len(s)):
  if "Position_" in s[i]:
    pos=s[i].replace("Position_","")
    if temppos1>int(pos):
      temp=32-temppos1+int(pos)
    elif temppos1<int(pos):
      temp=int(pos)-temppos1
    elif temppos1==int(pos):
      temp=32
    temppos1=int(pos)
    temppos2=temppos2+temp
  elif "Pitch_" in s[i]:
    if i+1==len(s):
      break
    pospitch=s[i].replace("Pitch_","")
    posduration=s[i+1].replace("Duration_","")
    tempdura=previous_dura[back_dura.index(posduration)]
    note = pretty_midi.Note(velocity=100, pitch=int(pospitch), start=float(temppos2*0.0625), end=(temppos2*0.0625)+float(tempdura))
    cello.notes.append(note)
cello_c_chord.instruments.append(cello)
cello_c_chord.write('/content/drive/MyDrive/s_melody_generation/output1.mid')

In [ ]:
from midi2audio import FluidSynth
from IPython.display import Audio
import os

midi_dir = os.path.join(os.path.dirname('/content/drive/MyDrive/s_melody_generation'), 's_melody_generation')
midi_path = os.path.join(midi_dir, 'output1.mid')
FluidSynth("font.sf2").midi_to_audio(midi_path, 'output1.wav')
Audio("output1.wav")